In [46]:
import pandas as pd
from ortools.linear_solver import pywraplp
import numpy as np

dataset = pd.read_csv("../Downloads/truc/faces.utf8.csv")
briefs = pd.read_csv("../Downloads/truc/briefs.utf8.csv")
briefs = briefs.values
tmp = np.array([6.319625, 2.083290, 4.578867, 2.754223, 1, 8.543064, 10.461262, 9.185119, 1.330685])
faces_cpm = np.matrix(dataset.iloc[:, 10:19], dtype=float)
cpm_matrix = np.eye(9, dtype=float)
cpm_row = tmp
cpm_tomult = cpm_row.T * cpm_matrix
faces_cpm *= cpm_tomult
max_cpm = faces_cpm.max(1)
dataset['rev_max'] = max_cpm
dataset = dataset.values
result = []
def get_week(week, data):
    array = []
    for i in range(0, len(data)):
        if data[i][0] == week:
            array.append(data[i])
    return (array)

dict_col = {}
dict_col["25_49csp"] = 12
dict_col["cspp"] = 15
dict_col["urb"] = 18
dict_col["all"] = 14
dict_col["f25_49rda"] = 17
dict_col["f25_49csp"] = 16
dict_col["15_24"] = 10
dict_col["25_49"] = 11
dict_col["25_49urb"] = 13

def get_max_week():
    return (briefs[len(briefs)-1][0])

def get_coeff(data, contactkey):
    if data[19] == 0:
        return (0)
    return (data[contactkey] * tmp[contactkey - 10] / data[19] * 100)

In [47]:
def solve_week(week_nb, result):
    brief = [[]]
    full_week = get_week(week_nb, briefs)
    data = dataset.tolist()
    for k in range (0, len(full_week)):
        brief[0] = full_week[k]
        contactkey = dict_col[brief[0][2]]
        solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

        faces = [[]] * len(data)
        objective = solver.Objective()
        for i in range(0, len(data)):
            faces[i] = solver.NumVar(0, 1, data[i][0])
            objective.SetCoefficient(faces[i], get_coeff(data[i], contactkey))
        objective.SetMinimization()

        constraints = [0] * len(brief)
        for i in range(0, len(brief)):
            constraints[i] = solver.Constraint(brief[i][3], solver.infinity())
            for j in range(0, len(data)):
                constraints[i].SetCoefficient(faces[j], data[j][contactkey])
                

        status = solver.Solve()
        ref = []
        if status == solver.OPTIMAL:
            nb_faces = 0
            re = []
            indexes = []
            for i in range (0, len(data)):
                if (faces[i].solution_value() > 0):
                    nb_faces += 1
                    indexes.append(i)
                    re.append(data[i][0])
            result.append(re)
            for i in reversed(indexes):
                data.pop(i)
        else:
            result.append(ref)
    return (result)

for i in range(1, get_max_week() + 1):
    result = solve_week(i, result)

In [48]:
def make_key(week, order):
    return '"(week-' + str(week) + ", order-" + str(order) + ')"'
def make_value(sol):
    ret = "["
    l = len(sol)
    for i in range(0, l):
        ret += '"'+str(sol[i])+'"'
        if i != l - 1:
            ret +=","
    ret += "]"
    return ret
def to_submit(sols):
    ret = "{"
    l = len(sols)
   # if l != len(briefs):
   #    return "Error. The number of briefs and number of results are not matching."
    for i in range(0, l):
        if len(sols[i]) != 0:
            ret += make_key(briefs[i][0], briefs[i][1]) + ":" + make_value(sols[i])
            if i != l -1:
                ret += ","
    if ret[-1] == ',':
        ret = ret[:-1]
    ret += "}"
    return ret

In [52]:
import json
s = to_submit(result)
test = json.JSONDecoder()
test.decode(s)
print(len(result))

232


In [53]:
f = open("answer.json", "w")
f.write(s)
f.close()